# Algorithmic Pokémon Teambuilding

This notebook accompanies the article and focuses on reproducible code and results.

- Data and format
- Candidate sets from usage
- Simulated self-play and caching
- Baseline logistic model
- Interaction model (synergy, counters)
- Visualizations and tables
- Best teams vs uniform meta
- Iterative counters and Nash equilibrium


In [ ]:
import sys

from poke_env.player.baselines import SimpleHeuristicsPlayer
from poke_env.teambuilder import ConstantTeambuilder, TeambuilderPokemon, Teambuilder
from sklearn.linear_model import LogisticRegression
import nashpy as nash
import requests
import json
from tqdm import tqdm
import os
import random
from poke_env.battle.status import Status
import numpy as np

from IPython.core.display import HTML

### Data: Smogon usage stats (Gen 1 OU, July 2025)

In [2]:
RANDOM_SEED = 42
FORMAT = "gen1ou"
TIME_PERIOD = "2025-07"

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

cache_path = f"cache/{TIME_PERIOD}/{FORMAT}-0.json"

if not os.path.exists(cache_path):
    data = requests.get(f"https://www.smogon.com/stats/{TIME_PERIOD}/chaos/{FORMAT}-0.json").json()

    os.makedirs(os.path.dirname(cache_path), exist_ok=True)

    with open(cache_path, "w+") as f:
        json.dump(data, f)
else:
    with open(cache_path, "r") as f:
        data = json.load(f)


### Candidate sets from usage

We keep Pokémon with ≥0.1% usage and build one set per Pokémon using its four most used moves.

In [3]:
def mon_to_teambuilder_pokemon(mon: str, data) -> TeambuilderPokemon:
    moves = data['data'][mon]['Moves']
    top_moves = sorted([m for m in moves if m != ""], key=lambda move: -moves[move])
    return TeambuilderPokemon(
        nickname=mon.lower(),
        species=mon.lower(),
        moves=[m.lower() for m in top_moves[:4]],
        ability="noability",
        nature="serious",
        evs=[252] * 6,
        item=None,
    )

sets = []
mon_to_idx = {}

USAGE_THRESHOLD = 0.001  # 0.1%

total_count = sum(mon_stats['Raw count'] for mon_stats in data['data'].values())
qualified_mons = [mon for mon, stats in data['data'].items() if stats['Raw count'] / total_count >= USAGE_THRESHOLD]
qualified_mons.sort(key=lambda m: -data['data'][m]['Raw count'])

for mon in qualified_mons:
    sets.append(mon_to_teambuilder_pokemon(mon, data))
    mon_to_idx[mon.lower()] = len(mon_to_idx)

print(f"Created {len(sets)} sets (>=0.1% usage).")


Created 47 sets (>=0.1% usage).


### Simulated self-play

Play random vs random teams to generate labeled matches; cache results to disk.

In [4]:
N_BATTLES = 50_000

player_a = SimpleHeuristicsPlayer(battle_format=FORMAT, max_concurrent_battles=100)
player_b = SimpleHeuristicsPlayer(battle_format=FORMAT, max_concurrent_battles=100)

battles_cache_dir = f"cache/{TIME_PERIOD}/{FORMAT}-battles/"
os.makedirs(battles_cache_dir, exist_ok=True)

n_battles_to_play = max(0, N_BATTLES - len(os.listdir(battles_cache_dir)))

for _ in tqdm(range(n_battles_to_play), desc="Simulating"):
    mons_a = random.sample(sets, 6)
    mons_b = random.sample(sets, 6)

    team_a = ConstantTeambuilder.join_team(mons_a)
    team_b = ConstantTeambuilder.join_team(mons_b)

    player_a.update_team(team_a)
    player_b.update_team(team_b)

    await player_a.battle_against(player_b)

    battle = list(player_a.battles.values())[0]

    result = {
        "team_a": team_a,
        "team_b": team_b,
        "result": battle.won,
        "a_fainted": sum(mon_obj.status == Status.FNT for mon_obj in battle.team.values()),
        "b_fainted": sum(mon_obj.status == Status.FNT for mon_obj in battle.opponent_team.values()),
    }

    with open(f"{battles_cache_dir}/{battle.battle_tag}.json", "w+") as f:
        json.dump(result, f)

    player_a.reset_battles()


Simulating: 0it [00:00, ?it/s]


### Baseline logistic regression

Fit additive strengths per Pokémon; export scores.

In [5]:
all_results = []

for file in tqdm(os.listdir(battles_cache_dir), desc="Loading results"):
    with open(f"{battles_cache_dir}/{file}", "r") as f:
        result = json.load(f)

        if result['result'] is None:
            continue

        all_results.append(result)

len(all_results)

Loading results: 100%|██████████| 50000/50000 [00:01<00:00, 40582.93it/s]


49581

In [6]:
x = np.zeros((len(all_results), len(mon_to_idx)))
y = np.zeros(len(all_results), dtype=np.int8)

for i, result in enumerate(all_results):
    if result['result'] is None:
        continue

    mons_a = Teambuilder.parse_packed_team(result['team_a'])
    mons_b = Teambuilder.parse_packed_team(result['team_b'])

    for mon in mons_a:
        x[i, mon_to_idx[mon.species]] += 1

    for mon in mons_b:
        x[i, mon_to_idx[mon.species]] -= 1

    y[i] = result['result']

In [7]:
baseline_lr = LogisticRegression(max_iter=1000)
baseline_lr.fit(np.concatenate([x, -x]), np.concatenate([y, 1-y]))

baseline_mon_to_score = {mon: baseline_lr.coef_[0][idx] for mon, idx in mon_to_idx.items()}

for mon, score in sorted(baseline_mon_to_score.items(), key=lambda x: x[1], reverse=True):
    print(mon, score)

alakazam 0.7144246902052549
zapdos 0.6830770223898542
starmie 0.5676573198695237
jolteon 0.493826090353679
articuno 0.40134084469397935
lapras 0.3930253486190372
tauros 0.3690722585706851
jynx 0.3463664915028059
raichu 0.30063246121515547
hypno 0.22160630362106318
gyarados 0.18016450356864677
moltres 0.16467555599050734
snorlax 0.1621407939886784
chansey 0.14987502932398528
ninetales 0.1136762873472259
nidoking 0.09645228001954848
clefable 0.09484972012400693
gengar 0.06050751113953537
electrode 0.04628124502468713
dragonite 0.035628075805313485
persian 0.03120750338281161
exeggutor 0.00810631401735352
rhydon 0.0016538029332805126
blastoise -0.008711657728743966
dugtrio -0.014122735064518362
charizard -0.01799854092577897
arcanine -0.07277510434899286
magmar -0.07983426997142726
cloyster -0.09819191405828014
golem -0.12293198885818082
dodrio -0.12409815709527644
kangaskhan -0.12842360623238863
slowbro -0.1772806680031323
machamp -0.19189165193670152
poliwrath -0.22982199438641432
kabut

### Interaction model

Pairwise features for within-team synergy and cross-team counters, trained with regularization.

In [8]:
interaction_idxs = {mon: i for mon, i in mon_to_idx.items()}

for mon in mon_to_idx:
    for mon2 in mon_to_idx:
        if mon >= mon2:
            continue

        interaction_idxs[(mon, mon2, "opp_team")] = len(interaction_idxs)
        interaction_idxs[(mon, mon2, "same_team")] = len(interaction_idxs)

def x_from_mons(mons_a, mons_b):
    x = np.zeros(len(interaction_idxs))

    for mon in mons_a:
        x[mon_to_idx[mon]] += 1

    for mon in mons_b:
        x[mon_to_idx[mon]] -= 1
    
    for mon1 in mons_a:
        for mon2 in mons_a:
            if mon1 >= mon2:
                continue
            
            x[interaction_idxs[(mon1, mon2, "same_team")]] += 1

    for mon1 in mons_b:
        for mon2 in mons_b:
            if mon1 >= mon2:
                continue
            
            x[interaction_idxs[(mon1, mon2, "same_team")]] -= 1

    for mon1 in mons_a:
        for mon2 in mons_b:
            if mon1 == mon2:
                continue
            elif mon1 < mon2:
                x[interaction_idxs[(mon1, mon2, "opp_team")]] += 1
            else:
                x[interaction_idxs[(mon2, mon1, "opp_team")]] -= 1

    return x

x = np.zeros((len(all_results), len(interaction_idxs)), dtype=np.int8)
y = np.zeros(len(all_results), dtype=np.int8)

for i, result in enumerate(all_results):
    if result['result'] is None:
        continue

    mons_a = Teambuilder.parse_packed_team(result['team_a'])
    mons_b = Teambuilder.parse_packed_team(result['team_b'])

    x[i] = x_from_mons([m.species for m in mons_a], [m.species for m in mons_b])
    y[i] = result['result']

In [9]:
x_in = x[:len(x) // 2]
y_in = y[:len(y) // 2]

x_out = x[len(x) // 2:]
y_out = y[len(y) // 2:]

c_to_score = {}
cs = [.0001, .0003, .001, .003, .01, .03, .1, .3, 1, 3, 10, 30, 100, 300]

for c in cs:
    lr = LogisticRegression(C=c, max_iter=1000)
    lr.fit(np.concatenate([x_in, -x_in]), np.concatenate([y_in, 1-y_in]))

    c_to_score[c] = lr.score(x_out, y_out)
    print(f"{c:8.3f}: {c_to_score[c]*100:5.2f}")

best_c = max(cs, key=lambda c: c_to_score[c])
assert (best_c != cs[0] and best_c != cs[-1])

interactions_lr = LogisticRegression(C=best_c, max_iter=1000)
interactions_lr.fit(np.concatenate([x, -x]), np.concatenate([y, 1-y]))

   0.000: 67.30
   0.000: 67.56
   0.001: 68.04
   0.003: 67.89
   0.010: 67.35
   0.030: 66.88
   0.100: 66.60
   0.300: 66.46
   1.000: 66.38
   3.000: 66.36
  10.000: 66.35
  30.000: 66.35
 100.000: 66.35
 300.000: 66.35


LogisticRegression(C=0.001, max_iter=1000)

In [10]:
interactions_mon_to_score = {}

for mon, idx in mon_to_idx.items():
    interactions_mon_to_score[mon] = interactions_lr.coef_[0][idx]

for i, (mon, score) in enumerate(sorted(interactions_mon_to_score.items(), key=lambda x: x[1], reverse=True)[:10]):
    print(f"{i+1:2d}. {mon.capitalize():11s}: {score*100:4.0f}")

    synergy = {}
    counters = {}
    countered_by = {}

    for key in interaction_idxs:
        if len(key) != 3:
            continue

        if (key[0] == mon) or (key[1] == mon):
            other_mon = key[0] if key[0] != mon else key[1]
            if key[2] == "same_team":
                synergy[other_mon] = interactions_lr.coef_[0][interaction_idxs[key]]
            elif key[0] == mon:
                counters[other_mon] = interactions_lr.coef_[0][interaction_idxs[key]]
                countered_by[other_mon] = -interactions_lr.coef_[0][interaction_idxs[key]]
            else:
                counters[other_mon] = -interactions_lr.coef_[0][interaction_idxs[key]]
                countered_by[other_mon] = interactions_lr.coef_[0][interaction_idxs[key]]

    sorted_synergy = sorted(synergy, key=lambda x: synergy[x], reverse=True)
    sorted_counter = sorted(counters, key=lambda x: counters[x], reverse=True)
    sorted_countered_by = sorted(countered_by, key=lambda x: countered_by[x], reverse=True)

    print("- Synergy:")
    for with_synergy in sorted_synergy[:5]:
        score = synergy[with_synergy]
        print(f"  {with_synergy.capitalize():11s}: {score*100:4.0f}")

    print("- Counters:")
    for counter in sorted_counter[:5]:
        score = counters[counter]
        print(f"  {counter.capitalize():11s}: {score*100:4.0f}")

    print("- Countered by:")
    for countered in sorted_countered_by[:5]:
        score = countered_by[countered]
        print(f"  {countered.capitalize():11s}: {score*100:4.0f}")



 1. Alakazam   :   28
- Synergy:
  Gengar     :    7
  Machamp    :    6
  Poliwrath  :    6
  Ninetales  :    6
  Kangaskhan :    6
- Counters:
  Raichu     :   12
  Tangela    :   11
  Pinsir     :   10
  Victreebel :   10
  Machamp    :    9
- Countered by:
  Snorlax    :    6
  Exeggutor  :    4
  Jynx       :    2
  Aerodactyl :    2
  Moltres    :    1
 2. Zapdos     :   26
- Synergy:
  Persian    :    7
  Jynx       :    7
  Sandslash  :    7
  Clefable   :    6
  Raichu     :    6
- Counters:
  Kabutops   :   14
  Magmar     :   10
  Slowbro    :   10
  Gyarados   :    9
  Hitmonlee  :    8
- Countered by:
  Rhydon     :   21
  Golem      :   17
  Jolteon    :   15
  Alakazam   :    3
  Raichu     :    2
 3. Starmie    :   22
- Synergy:
  Venusaur   :    6
  Alakazam   :    5
  Articuno   :    5
  Tauros     :    5
  Dugtrio    :    5
- Counters:
  Raticate   :    9
  Charizard  :    8
  Dragonite  :    8
  Articuno   :    8
  Kingler    :    8
- Countered by:
  Jolteon    :   

In [11]:
interactions_mon_to_data = {}

for mon, idx in mon_to_idx.items():
    interactions_mon_to_data[mon] = {"theta": float(interactions_lr.coef_[0][idx]), "alpha": {}, "beta": {}}

    for key in interaction_idxs:
        if len(key) != 3:
            continue

        if not(key[0] == mon) and not(key[1] == mon):
            continue

        other_mon = key[0] if key[0] != mon else key[1]

        if key[2] == "same_team":
            interactions_mon_to_data[mon]["alpha"][other_mon] = float(interactions_lr.coef_[0][interaction_idxs[key]])
        else:
            if key[0] == mon:
                interactions_mon_to_data[mon]["beta"][other_mon] = float(interactions_lr.coef_[0][interaction_idxs[key]])
            else:
                interactions_mon_to_data[mon]["beta"][other_mon] = float(-interactions_lr.coef_[0][interaction_idxs[key]])


### Evaluate best baseline team

In [12]:
best_baseline_mons = sorted(baseline_mon_to_score, key=lambda k: baseline_mon_to_score[k], reverse=True)[:6]

player_a.reset_battles()

for _ in tqdm(range(500), desc="Baseline vs random"):
    random.shuffle(best_baseline_mons)
    team_a = ConstantTeambuilder.join_team([sets[mon_to_idx[mon]] for mon in best_baseline_mons])
    player_a.update_team(team_a)

    mons_b = random.sample(sets, 6)
    team_b = ConstantTeambuilder.join_team(mons_b)
    player_b.update_team(team_b)

    await player_a.battle_against(player_b)

uniform_baseline_observed = player_a.win_rate
uniform_baseline_observed

Baseline vs random: 100%|██████████| 500/500 [00:22<00:00, 21.87it/s]


0.942

In [13]:
best_baseline_mons

['zapdos', 'jolteon', 'alakazam', 'lapras', 'starmie', 'articuno']

### Best counter to the baseline team

In [14]:
def win_rate_a_vs_b(a_mons, b_mons, lr):
    x = x_from_mons(a_mons, b_mons)
    return lr.predict_proba(x.reshape(1, -1))[0, 1]


def build_optimal_counter_team(team, lr):
    current_team = sorted(mon_to_idx, key=lambda m: -lr.coef_[0, mon_to_idx[m]])[:6]
    current_proba = win_rate_a_vs_b(current_team, team, lr)

    position = 0

    while position != 6:
        next_position = position + 1

        for mon in mon_to_idx:
            if mon in current_team:
                continue

            new_team = current_team[:]
            new_team[position] = mon
            new_proba = win_rate_a_vs_b(new_team, team, lr)

            if new_proba > current_proba:
                current_team, current_proba = new_team, new_proba
                next_position = 0

        position = next_position

    return current_team


best_baseline_counter = build_optimal_counter_team(best_baseline_mons, interactions_lr)
interaction_vs_baseline_predicted = win_rate_a_vs_b(best_baseline_counter, best_baseline_mons, interactions_lr)

print("best counter team:", best_baseline_counter)
print("predicted win rate:", interaction_vs_baseline_predicted)

player_a.reset_battles()
for _ in tqdm(range(500), desc="Interaction vs baseline"):
    random.shuffle(best_baseline_mons)
    random.shuffle(best_baseline_counter)

    team_a = ConstantTeambuilder.join_team([sets[mon_to_idx[mon]] for mon in best_baseline_counter])
    team_b = ConstantTeambuilder.join_team([sets[mon_to_idx[mon]] for mon in best_baseline_mons])

    player_a.update_team(team_a)
    player_b.update_team(team_b)

    await player_a.battle_against(player_b)

print("actual win rate:", player_a.win_rate)
interaction_vs_baseline_observed = player_a.win_rate

best counter team: ['alakazam', 'zapdos', 'raichu', 'jolteon', 'jynx', 'rhydon']
predicted win rate: 0.6513586681389703


Interaction vs baseline: 100%|██████████| 500/500 [00:20<00:00, 24.22it/s]

actual win rate: 0.682


In [15]:
def win_rate_a_vs_bs(a_mons, b_teams, lr):
    X = np.vstack([x_from_mons(a_mons, b_mons) for b_mons in b_teams])
    return float(lr.predict_proba(X)[:, 1].mean())


def build_optimal_counter_teams(teams, lr):
    current_team = sorted(mon_to_idx, key=lambda m: -lr.coef_[0, mon_to_idx[m]])[:6]
    current_proba = win_rate_a_vs_bs(current_team, teams, lr)

    position = 0

    while position != 6:
        next_position = position + 1

        for mon in mon_to_idx:
            if mon in current_team:
                continue

            new_team = current_team[:]
            new_team[position] = mon
            new_proba = win_rate_a_vs_bs(new_team, teams, lr)

            if new_proba > current_proba:
                current_team, current_proba = new_team, new_proba
                next_position = 0

        position = next_position

    return current_team


sample_teams = [random.sample(sorted(mon_to_idx), 6) for _ in range(100)]

optimal_uniform_team = build_optimal_counter_teams(sample_teams, interactions_lr)

team_a = ConstantTeambuilder.join_team([sets[mon_to_idx[mon]] for mon in optimal_uniform_team])
interaction_uniform_predicted = win_rate_a_vs_bs(optimal_uniform_team, sample_teams, interactions_lr)

print("best uniform team:", optimal_uniform_team)
print("predicted win rate:", interaction_uniform_predicted)

player_a.reset_battles()

for _ in tqdm(range(500), desc="Optimal vs random"):
    random.shuffle(optimal_uniform_team)

    team_a = ConstantTeambuilder.join_team([sets[mon_to_idx[mon]] for mon in optimal_uniform_team])
    player_a.update_team(team_a)
    
    mons_b = random.sample(sets, 6)
    team_b = ConstantTeambuilder.join_team(mons_b)
    player_b.update_team(team_b)

    await player_a.battle_against(player_b)

interaction_uniform_observed = player_a.win_rate
print("actual win rate:", interaction_uniform_observed)

assert set(optimal_uniform_team) == set(['alakazam', 'zapdos', 'starmie', 'jolteon', 'tauros', 'raichu'])

best uniform team: ['alakazam', 'zapdos', 'starmie', 'jolteon', 'tauros', 'raichu']
predicted win rate: 0.928296520874682


Optimal vs random: 100%|██████████| 500/500 [00:17<00:00, 29.02it/s]

actual win rate: 0.946


### Nash equilibrium over reduced team pool

In [16]:
current_pool = [optimal_uniform_team]
print(optimal_uniform_team)
current_pool_set = {','.join(sorted(t)) for t in current_pool}

while True:
    if len(current_pool) > 100:
        break

    best_counter = build_optimal_counter_team(current_pool[-1], interactions_lr)
    best_counter_as_string = ','.join(sorted(best_counter))

    print(best_counter)

    if best_counter_as_string in current_pool_set:
        break

    current_pool_set.add(best_counter_as_string)
    current_pool.append(best_counter)


['jolteon', 'alakazam', 'zapdos', 'starmie', 'raichu', 'tauros']
['alakazam', 'tauros', 'rhydon', 'jolteon', 'zapdos', 'nidoking']
['alakazam', 'tauros', 'rhydon', 'jynx', 'exeggutor', 'jolteon']
['tauros', 'alakazam', 'jynx', 'raichu', 'articuno', 'jolteon']
['alakazam', 'tauros', 'zapdos', 'jolteon', 'jynx', 'raichu']
['alakazam', 'tauros', 'rhydon', 'jolteon', 'zapdos', 'nidoking']


In [17]:
payoff_matrix = np.zeros((len(current_pool), len(current_pool)))

for i, team1 in enumerate(current_pool):
    for j, team2 in enumerate(current_pool):
        if i >= j:
            continue

        p = win_rate_a_vs_b(team1, team2, interactions_lr) / 2 + (1 - win_rate_a_vs_b(team2, team1, interactions_lr)) / 2
        payoff_matrix[i,j] = 2 * p - 1
        payoff_matrix[j,i] = 1 - 2 * p

interaction_metagame = nash.Game(payoff_matrix)
nash_equilibrium = list(interaction_metagame.support_enumeration())

print(payoff_matrix)
print(nash_equilibrium)

for team, value in zip(current_pool, nash_equilibrium[0][0]):
    if value > 0:
        print(team, value)

[[ 0.         -0.07267954 -0.03791561  0.09043282 -0.02559196]
 [ 0.07267954  0.         -0.11589418 -0.0041866   0.04924074]
 [ 0.03791561  0.11589418  0.         -0.09045432 -0.0193335 ]
 [-0.09043282  0.0041866   0.09045432  0.         -0.09841992]
 [ 0.02559196 -0.04924074  0.0193335   0.09841992  0.        ]]
[(array([0.        , 0.10480654, 0.26693317, 0.        , 0.62826029]), array([0.        , 0.10480654, 0.26693317, 0.        , 0.62826029]))]
['alakazam', 'tauros', 'rhydon', 'jolteon', 'zapdos', 'nidoking'] 0.10480653724830724
['alakazam', 'tauros', 'rhydon', 'jynx', 'exeggutor', 'jolteon'] 0.266933170685845
['alakazam', 'tauros', 'zapdos', 'jolteon', 'jynx', 'raichu'] 0.6282602920658478


In [18]:
win_rates = {
    "interaction_vs_baseline_observed": interaction_vs_baseline_observed,
    "interaction_vs_baseline_predicted": interaction_vs_baseline_predicted,
    "interaction_uniform_predicted": interaction_uniform_predicted,
    "interaction_uniform_observed": interaction_uniform_observed,
    "uniform_baseline_observed": uniform_baseline_observed,
    
}

with open("teambuilding_gen1ou_data.json", "w+") as f:
    json.dump(
        {
            "win_rates": win_rates,
            "baseline_model": baseline_mon_to_score,
            "interactions_model": interactions_mon_to_data,
        },
        f
    )

# Showdown formatted teams for export

In [19]:
# No‑interaction best team (also used as t0): Alakazam, Zapdos, Starmie, Jolteon, Articuno, Lapras
# Counter-team to baseline (3.2.3): Alakazam, Zapdos, Raichu, Jolteon, Jynx, Rhydon
# Uniform-optimized team (3.2.4): Alakazam, Zapdos, Tauros, Jolteon, Starmie, Raichu
# Nash mix team #1 (62.8%) (3.2.5): Alakazam, Tauros, Zapdos, Jolteon, Jynx, Raichu
# Nash mix team #2 (26.7%) (3.2.5): Alakazam, Tauros, Rhydon, Jynx, Exeggutor, Jolteon
# Nash mix team #3 (10.5%) (3.2.5): Alakazam, Tauros, Rhydon, Jolteon, Zapdos, Nidoking

teams = {
    "no_interaction_best_team": ['alakazam', 'zapdos', 'starmie', 'jolteon', 'articuno', 'lapras'],
    "counter_to_baseline": ['alakazam', 'zapdos', 'raichu', 'jolteon', 'jynx', 'rhydon'],
    "uniform_optimized_team": ['alakazam', 'zapdos', 'tauros', 'jolteon', 'starmie', 'raichu'],
    "nash_mix_team_1": ['alakazam', 'tauros', 'zapdos', 'jolteon', 'jynx', 'raichu'],
    "nash_mix_team_2": ['alakazam', 'tauros', 'rhydon', 'jynx', 'exeggutor', 'jolteon'],
    "nash_mix_team_3": ['alakazam', 'tauros', 'rhydon', 'jolteon', 'zapdos', 'nidoking']
}

for team_name, team in teams.items():
    mons_str = []

    for mon in team:
        mon_str = f"{mon.capitalize()}\nAbility: No ability"
        moves_str = "\n- ".join([m.capitalize() for m in sets[mon_to_idx[mon]].moves])
        mon_str += "\n- " + moves_str
        mons_str.append(mon_str)

    mons_str = "\n\n".join(mons_str)
    print(team_name)
    print()
    print(mons_str)
    print()

no_interaction_best_team

Alakazam
Ability: No ability
- Psychic
- Recover
- Thunderwave
- Seismictoss

Zapdos
Ability: No ability
- Drillpeck
- Thunderbolt
- Thunderwave
- Agility

Starmie
Ability: No ability
- Recover
- Thunderwave
- Psychic
- Blizzard

Jolteon
Ability: No ability
- Thunderbolt
- Thunderwave
- Doublekick
- Pinmissile

Articuno
Ability: No ability
- Blizzard
- Hyperbeam
- Agility
- Icebeam

Lapras
Ability: No ability
- Blizzard
- Thunderbolt
- Sing
- Confuseray

counter_to_baseline

Alakazam
Ability: No ability
- Psychic
- Recover
- Thunderwave
- Seismictoss

Zapdos
Ability: No ability
- Drillpeck
- Thunderbolt
- Thunderwave
- Agility

Raichu
Ability: No ability
- Thunderbolt
- Surf
- Thunderwave
- Bodyslam

Jolteon
Ability: No ability
- Thunderbolt
- Thunderwave
- Doublekick
- Pinmissile

Jynx
Ability: No ability
- Lovelykiss
- Psychic
- Blizzard
- Rest

Rhydon
Ability: No ability
- Earthquake
- Rockslide
- Bodyslam
- Substitute

uniform_optimized_team

Alakazam
Abil